In [1]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import tensorflow as tensor

In [129]:
dataset = pd.read_csv(r'news_articles.tsv', delimiter='\t', quoting=3, encoding='ISO-8859-1')
article = dataset.iloc[:,1]
topic = dataset.iloc[:,2]

In [3]:
master_word_list=[]
master_article=''
for j in range(len(article)):
    master_article+=article[j]

In [29]:
len(article)

256

In [105]:
stop_words=set(stopwords.words('english'))
#master_article=master_article.replace('',"'")
master_word_list=master_article.lower().split()
#word_tokens=word_tokenize(master_article.lower())
master_filtered_sentence=word_tokenize(master_article.lower())
#master_filtered_sentence=[w for w in word_tokens if not w in stop_words]
master_tagged_words=pos_tag(master_filtered_sentence)
#master_nouns=[word for word, pos in master_tagged_words\
              #if(pos == 'NN' or pos == 'NNP' or pos == 'NNS')]
#master_nouns[:] = [noun for noun in master_nouns if (noun != '\x94' and noun !='%')]
master_word_freq=[]

for k in range(len(master_filtered_sentence)):
        master_word_freq.append(master_word_list.count(master_filtered_sentence[k]))
idf=[]
for i in range(len(master_filtered_sentence)):
    score=0
    for j in range(len(article)):
        if(master_filtered_sentence[i] in article[j]):
            score+=1
    idf.append(score)    
dict_article={
      'data':master_filtered_sentence,
      'freq':master_word_freq,
      'idf':idf
     }  
columns=['data','freq','idf']     
master_set=pd.DataFrame(dict_article, columns=columns)
master_set=master_set.loc[master_set['freq']!=0]
master_set=master_set.loc[master_set['idf']!=0]
master_set=master_set.drop_duplicates().reset_index()
master_set=master_set.drop(['index'],axis=1)
master_set.iloc[:,1]=master_set.iloc[:,1].values.astype(np.float64)
master_set.iloc[:,2]=master_set.iloc[:,2].values.astype(np.float64)
tot=np.sum(master_set.iloc[:,1])
vocab=master_set.iloc[:,0]
master_set
vocab

0                  federal
1                  reserve
2                     bank
3                       of
4                president
5                        ,
6                      who
7                    votes
8                       on
9                   policy
10                    this
11                    year
12                    said
13                  recent
14             encouraging
15                economic
16                    data
17                      is
18                  enough
19                      to
20                  change
21                     the
22                    pace
23                 central
24                   asset
25               purchases
26                       .
27                      it
28                   would
29                    have
               ...        
8477               tangles
8478                 rooms
8479                 segue
8480            quick-draw
8481         long-distance
8482             piggyback
8

In [106]:
final_data=[0]*8509
for l in range (len(article)):
    article_data=article[l]
    stop_words=set(stopwords.words('english'))
    wordlist=article_data.lower().split()
    word_tokens=word_tokenize(article_data.lower())
    nouns = [w for w in word_tokens if not w in stop_words]
    tagged_word=pos_tag(nouns)
    #nouns=[word for word,pos in tagged_word\
            #if (pos == 'NN' or pos == 'NNP' or pos == 'NNS')]
    #nouns[:] = [noun for noun in nouns if (noun != '\x94' and noun !='%')]
    word_freq = []
    for i in range(len(nouns)):
        word_freq.append(wordlist.count(nouns[i]))
    dict_data=dict(zip(nouns,word_freq))
    tf=0.0
    sum=0.0
    presence=[]
    tf_idf=0.0
    pmi=0.0
    dict_data={k:v for k,v in dict_data.items() if v!=0}
    for master in range(len(master_set)):
        if(master_set.iloc[master,0] in dict_data):
            tf+=np.log(dict_data.get(master_set.iloc[master,0])/master_set.iloc[master,1])
            #tf+=np.log(master_set.iloc[master,1])
            sum+=dict_data.get(master_set.iloc[master,0])
            presence.append(1)
        else:
            presence.append(0)
    for master in range(len(master_set)):
        if(master_set.iloc[master,0] in dict_data):
            tf_idf+=(dict_data.get(master_set.iloc[master,0])/sum)*np.log(len(article)/master_set.iloc[master,2])
    #for a in range(len(master_set)):
        #for b in range(len(master_set)):
            #if(master_set.iloc[b,0]!=master_set.iloc[a,0] and master_set.iloc[b,0] in dict_data and master_set.iloc[a,0] in dict_data):
                #pmi+=np.log(1/(master_set.iloc[b,1]*master_set.iloc[a,1]))
    presence.insert(0,np.abs(tf/sum))
    presence.insert(1,tf_idf/sum)
    #presence.insert(2,pmi)
    final_data=np.column_stack((final_data,presence))

In [107]:
final_data.shape

(8509, 257)

In [108]:
final_data=final_data[:,1:257]

In [109]:
master_column=master_set.iloc[:,0].tolist()
master_column.insert(0,'tf')
master_column.insert(1,'tf-idf')
#master_column.insert(2,'pmi')
master_column

['tf',
 'tf-idf',
 'federal',
 'reserve',
 'bank',
 'of',
 'president',
 ',',
 'who',
 'votes',
 'on',
 'policy',
 'this',
 'year',
 'said',
 'recent',
 'encouraging',
 'economic',
 'data',
 'is',
 'enough',
 'to',
 'change',
 'the',
 'pace',
 'central',
 'asset',
 'purchases',
 '.',
 'it',
 'would',
 'have',
 'been',
 'nice',
 'from',
 'my',
 'perspective',
 'had',
 'we',
 'started',
 'at',
 'a',
 'faster',
 'in',
 'television',
 'interview',
 'with',
 'paris',
 'referring',
 'fed',
 'tapering',
 'its',
 'stimulus',
 'programme',
 'given',
 'fact',
 'that',
 'embarked',
 'measured',
 'reductions',
 'important',
 'give',
 'some',
 'certainty',
 'or',
 'least',
 'clarity',
 'markets',
 'what',
 'doing',
 'ok',
 'continue',
 '10',
 'billion',
 'hurdle',
 'rate',
 'for',
 'pretty',
 'high',
 'either',
 'direction',
 'trying',
 'determine',
 'whether',
 'economy',
 'has',
 'shown',
 'signs',
 'weakness',
 'because',
 'severe',
 'winter',
 'weather',
 'fundamental',
 'obstacles',
 'growth',

In [110]:
master_data=pd.DataFrame(data=final_data.transpose(),columns=master_column)

In [111]:
topic=pd.get_dummies(topic)

In [112]:
topic

,0,1,2,3
0,1,0,0,0
1,1,0,0,0
2,1,0,0,0
3,1,0,0,0
4,1,0,0,0
5,1,0,0,0
6,1,0,0,0
7,1,0,0,0
8,1,0,0,0
9,1,0,0,0


In [113]:
len(master_data)

256

In [114]:
from sklearn.model_selection import train_test_split
tr_features,ts_features,tr_labels,ts_labels=train_test_split(master_data, topic, test_size=0.2, random_state=1)

In [115]:
ts_labels
#ts_labels=ts_labels.reshape(ts_labels.size,1)

,0,1,2,3
183,0,0,1,0
34,1,0,0,0
185,0,0,0,1
233,0,0,0,1
216,0,0,0,1
181,0,0,1,0
160,0,0,1,0
27,1,0,0,0
145,0,0,1,0
187,0,0,0,1


In [116]:
training_epochs=28
n_neurons_h1=60
n_neurons_h2=60
learning_rate=0.01
n_features = tr_features.shape[1]
n_classes = 4

In [117]:
ts_labels

,0,1,2,3
183,0,0,1,0
34,1,0,0,0
185,0,0,0,1
233,0,0,0,1
216,0,0,0,1
181,0,0,1,0
160,0,0,1,0
27,1,0,0,0
145,0,0,1,0
187,0,0,0,1


In [118]:
X = tensor.placeholder(tensor.float32, [None, n_features], name='features')
Y = tensor.placeholder(tensor.float32, [None,n_classes], name='labels')
#X_test= tensor.placeholder(tensor.float32, [6, n_features], name='ts_features')
#Y_test= tensor.placeholder(tensor.float32, [6,], name='ts_labels')
keep_prob=tensor.placeholder(tensor.float32,name='drop_prob')

In [119]:
tr_features

,tf,tf-idf,federal,reserve,bank,of,president,",",who,votes,...,distractingly,repetitive,refining,sessions,repopulated,high-level,ranked,reacting,10th,17th
35,1.199778,0.008063,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,1.474195,0.006718,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29,1.287883,0.005722,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,1.786157,0.015939,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
159,1.698005,0.007587,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
112,1.966268,0.021503,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
122,2.264092,0.060333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
158,2.866368,0.125633,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
154,1.850984,0.007974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
232,0.823593,0.005088,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [120]:
W1 = tensor.Variable(tensor.truncated_normal([n_features, n_neurons_h1], mean=0, stddev=1 / np.sqrt(n_features)), name='weights1')
b1 = tensor.Variable(tensor.truncated_normal([n_neurons_h1],mean=0, stddev=1 / np.sqrt(n_features)), name='biases1')

In [121]:
y1 = tensor.matmul(X, W1)+b1
drop_out_layer1 = tensor.nn.dropout(y1, keep_prob)

In [122]:
#network parameters(weights and biases) are set and initialized(Layer2)
W2 = tensor.Variable(tensor.random_normal([n_neurons_h1, n_neurons_h2],mean=0,stddev=1/np.sqrt(n_features)),name='weights2')
b2 = tensor.Variable(tensor.random_normal([n_neurons_h2],mean=0,stddev=1/np.sqrt(n_features)),name='biases2')
#activation function(sigmoid)
y2 = tensor.nn.relu(tensor.matmul(y1,W2)+b2)
drop_out_layer2 = tensor.nn.dropout(y2, keep_prob)

In [123]:
#output layer weights and biasies
Wo = tensor.Variable(tensor.random_normal([n_neurons_h2, n_classes], mean=0, stddev=1/np.sqrt(n_features)), name='weightsOut')
bo = tensor.Variable(tensor.random_normal([n_classes], mean=0, stddev=1/np.sqrt(n_features)), name='biasesOut')
#activation function(softmax)
a = tensor.nn.softmax((tensor.matmul(y2, Wo) + bo), name='activationOutputLayer')

In [124]:
#cost function
cross_entropy = tensor.reduce_mean(tensor.nn.softmax_cross_entropy_with_logits(labels=Y,logits=a))
#optimizer
train_step = tensor.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

In [125]:
#compare predicted value from network with the expected value/target
correct_prediction = tensor.equal(tensor.argmax(a, 1), tensor.argmax(Y, 1))
#accuracy determination
accuracy = tensor.reduce_mean(tensor.cast(correct_prediction, tensor.float32), name="Accuracy")

In [126]:
# initialization of all variables
initial = tensor.global_variables_initializer()

#creating a session
with tensor.Session() as sess:
    sess.run(initial)
    writer = tensor.summary.FileWriter(r"C:\Users\Anurag\OneDrive\NLP\TrainResults")
    writer.add_graph(sess.graph)
    merged_summary = tensor.summary.merge_all()
    sess.run(train_step, feed_dict={X:tr_features , Y:tr_labels,keep_prob:0.5})
    y_pred = sess.run(tensor.argmax(a, 1), feed_dict={X: ts_features,keep_prob:1.0})
    #y_true = sess.run(tensor.convert_to_tensor(ts_labels, np.float32))
    y_true = sess.run(tensor.argmax(ts_labels, 1))
    acc = sess.run([accuracy], feed_dict={X: ts_features, Y: ts_labels,keep_prob:1.0})
    #writer.add_summary(summary, epoch)
    print('acc',acc)
    print ('---------------')
    print(y_pred, y_true)

acc [0.21153846]
---------------
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1] [2 0 3 3 3 2 2 0 2 3 3 1 0 3 0 2 3 0 2 3 1 1 3 3 3 1 0 3 2 3 1 0 1 1 1 3 0
 3 3 3 3 3 1 0 0 3 2 0 3 1 0 1]


In [127]:
correct_prediction

<tf.Tensor 'Equal_2:0' shape=(?,) dtype=bool>

In [128]:
topic

,0,1,2,3
0,1,0,0,0
1,1,0,0,0
2,1,0,0,0
3,1,0,0,0
4,1,0,0,0
5,1,0,0,0
6,1,0,0,0
7,1,0,0,0
8,1,0,0,0
9,1,0,0,0


In [130]:
from sklearn.preprocessing import LabelEncoder
labelEncoder=LabelEncoder()
topic=labelEncoder.fit_transform(topic)

In [131]:
classifier = MultinomialNB(alpha=0.0001)
from sklearn import metrics, cross_validation
predicted=cross_validation.cross_val_predict(classifier, master_data, topic, cv=4)
print (metrics.accuracy_score(topic,predicted))
print (metrics.classification_report(topic,predicted))

0.8984375
             precision    recall  f1-score   support

          0       0.93      0.89      0.91        56
          1       0.96      0.96      0.96        67
          2       0.92      0.77      0.84        61
          3       0.82      0.96      0.88        72

avg / total       0.90      0.90      0.90       256



In [134]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer(vocabulary=vocab)
X_idf=vectorizer.fit_transform(article)

In [135]:
classifier = MultinomialNB(alpha=0.001)
from sklearn import metrics, cross_validation
predicted=cross_validation.cross_val_predict(classifier, X_idf, topic, cv=4)
print (metrics.accuracy_score(topic,predicted))
print (metrics.classification_report(topic,predicted))

0.90234375
             precision    recall  f1-score   support

          0       0.86      0.89      0.88        56
          1       0.97      0.96      0.96        67
          2       0.91      0.80      0.85        61
          3       0.87      0.94      0.91        72

avg / total       0.90      0.90      0.90       256



In [51]:
from sklearn.model_selection import train_test_split
tr_features,ts_features,tr_labels,ts_labels=train_test_split(X_idf, topic, test_size=0.2, random_state=1)

In [61]:
ts_labels

,b,e,m,t
24,0,0,0,1
17,0,0,1,0
19,0,0,1,0
20,0,0,1,0
14,0,0,1,0
3,1,0,0,0


In [52]:
training_epochs=28
n_neurons_h1=60
n_neurons_h2=60
learning_rate=0.01
n_features = tr_features.shape[1]
n_classes = 4

In [53]:
X = tensor.placeholder(tensor.float32, [None, n_features], name='features')
Y = tensor.placeholder(tensor.float32, [None,n_classes], name='labels')
#X_test= tensor.placeholder(tensor.float32, [6, n_features], name='ts_features')
#Y_test= tensor.placeholder(tensor.float32, [6,], name='ts_labels')
keep_prob=tensor.placeholder(tensor.float32,name='drop_prob')

In [54]:
W1 = tensor.Variable(tensor.truncated_normal([n_features, n_neurons_h1], mean=0, stddev=1 / np.sqrt(n_features)), name='weights1')
b1 = tensor.Variable(tensor.truncated_normal([n_neurons_h1],mean=0, stddev=1 / np.sqrt(n_features)), name='biases1')
y1 = tensor.matmul(X, W1)+b1
drop_out_layer1 = tensor.nn.dropout(y1, keep_prob)

In [55]:
#network parameters(weights and biases) are set and initialized(Layer2)
W2 = tensor.Variable(tensor.random_normal([n_neurons_h1, n_neurons_h2],mean=0,stddev=1/np.sqrt(n_features)),name='weights2')
b2 = tensor.Variable(tensor.random_normal([n_neurons_h2],mean=0,stddev=1/np.sqrt(n_features)),name='biases2')
#activation function(sigmoid)
y2 = tensor.nn.relu(tensor.matmul(y1,W2)+b2)
#drop_out_layer2 = tensor.nn.dropout(y2, keep_prob)

In [56]:
#output layer weights and biasies
Wo = tensor.Variable(tensor.random_normal([n_neurons_h2, n_classes], mean=0, stddev=1/np.sqrt(n_features)), name='weightsOut')
bo = tensor.Variable(tensor.random_normal([n_classes], mean=0, stddev=1/np.sqrt(n_features)), name='biasesOut')
#activation function(softmax)
a = tensor.nn.softmax((tensor.matmul(y2, Wo) + bo), name='activationOutputLayer')

In [57]:
#cost function
cross_entropy = tensor.reduce_mean(tensor.nn.softmax_cross_entropy_with_logits(labels=Y,logits=a))
#optimizer
train_step = tensor.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

In [58]:
#compare predicted value from network with the expected value/target
correct_prediction = tensor.equal(tensor.argmax(a, 1), tensor.argmax(Y, 1))
#accuracy determination
accuracy = tensor.reduce_mean(tensor.cast(correct_prediction, tensor.float32), name="Accuracy")

In [59]:
# initialization of all variables
initial = tensor.global_variables_initializer()

#creating a session
with tensor.Session() as sess:
    sess.run(initial)
    writer = tensor.summary.FileWriter(r"C:\Users\Anurag\OneDrive\NLP\TrainResults")
    writer.add_graph(sess.graph)
    merged_summary = tensor.summary.merge_all()
    sess.run(train_step, feed_dict={X:tr_features , Y:tr_labels,keep_prob:0.5})
    y_pred = sess.run(tensor.argmax(a, 1), feed_dict={X: ts_features,keep_prob:1.0})
    #y_true = sess.run(tensor.convert_to_tensor(ts_labels, np.float32))
    y_true = sess.run(tensor.argmax(ts_labels, 1))
    acc = sess.run([accuracy], feed_dict={X: ts_features, Y: ts_labels,keep_prob:1.0})
    #writer.add_summary(summary, epoch)
    print('acc',acc)
    print ('---------------')
    print(y_pred, y_true)

ValueError: setting an array element with a sequence.